# Swyft Simulator

In this notebook, we will demonstrate how to generate training data using the `swyft.Simulator` class.

## Setup

First some imports.

In [13]:
import numpy as np
from scipy import stats
import pylab as plt
import torch
import swyft

## Definition of a simulator

The simulator defines a forward model for generating samples.  This happens in the `forward` method, where we populate the `trace` with sampling instructions for the relevant variables.

In [18]:
class Simulator(swyft.Simulator):
    def __init__(self):
        super().__init__()
        self.on_after_forward = swyft.to_numpy32

    def forward(self, trace):
        z = trace.sample('z', lambda: np.random.rand(1))
        x = trace.sample('x', lambda z: z + np.random.randn(1)*0.1, z)
        
sim = Simulator()

### Generating training data

It is now simple to generate training data.

In [19]:
samples = sim.sample(10000)

100%|██████████| 10000/10000 [00:00<00:00, 50832.35it/s]


Notice that the generated samples are single-precision, which is caused by the optional `on_after_forward` hook.

In [21]:
samples[0:3]

Samples({'z': array([[0.18097152],
       [0.6321205 ],
       [0.42921144]], dtype=float32), 'x': array([[0.24864887],
       [0.6349443 ],
       [0.33088908]], dtype=float32)})

### Why a simulator class?

The simulator class defines the computational graph of the simulator model.  Sample statements in the simulator are lazily evaluated. That means that only those components in the calculational graph are evaluated that are necessary for a given task.

As an example, we can use this to only generate samples for one of the variables.

In [22]:
sim.sample(targets = ['z'])

Sample({'z': array([0.9259025], dtype=float32)})

Or we can generate sample that are conditioned on certain variables.

In [23]:
sim.sample(conditions = {'z': np.array([1.])})

Sample({'z': array([1.], dtype=float32), 'x': array([1.1271212], dtype=float32)})